# Enriqueciendo Nuestros Datos con IA: Generación de Historias para Personajes de Dragon Ball usando Google Gemini

**Objetivo:** Aprender a utilizar la API de Google Gemini para generar texto creativo y actualizar la base de datos con esta nueva información.

---

## 1. Introducción a la IA Generativa y Google Gemini

La **Inteligencia Artificial Generativa** es una rama de la IA que se enfoca en la creación de contenido nuevo y original, como texto, imágenes, música y más. A diferencia de la IA tradicional que analiza datos existentes, la IA generativa puede producir resultados completamente nuevos.

**Google Gemini** es una familia de modelos de IA generativa de Google que es multimodal, lo que significa que puede entender y procesar diferentes tipos de información como texto, código, imágenes y video.

Para usar la API de Gemini, necesitamos una **API Key**, que es una clave única que nos identifica como usuarios y nos permite acceder a la API.

## 2. Obtención de una API Key de Google Gemini

Para obtener tu API Key gratuita, sigue estos pasos:

1.  Ve a [Google AI Studio](https://aistudio.google.com/).
2.  Inicia sesión con tu cuenta de Google.
3.  Haz clic en **"Get API key"** en la esquina inferior izquierda.
4.  Crea una nueva clave de API en la esquina superior derecha para un proyecto existente o en uno nuevo.
5.  Copia tu clave de API. ¡Trátala como una contraseña y no la compartas!

**Importante:** Nunca subas tu clave de API a repositorios públicos como GitHub.

## 3. Configuración del Entorno

Primero, instalaremos la librería de Python de Google Gemini.

In [1]:
%pip install -q -U google-generativeai 
%pip install -q python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import google.generativeai as genai
import sqlite3
import os
from dotenv import load_dotenv

## 4. Configuración de la API Key

Vamos a configurar la API key de una manera segura. Reemplaza `"YOUR_API_KEY"` con la clave que obtuviste.

In [4]:
# Cargar variables desde el archivo .env
load_dotenv()

# Leer la API key desde el entorno
API_KEY = os.getenv("GOOGLE_AI_STUDIO_APIKEY")

if not API_KEY:
    raise ValueError("⚠️ No se encontró la variable GOOGLE_API_KEY en el archivo .env")

# Configurar Google Gemini
genai.configure(api_key=API_KEY)
print("✅ API Key cargada y configurada correctamente.")


✅ API Key cargada y configurada correctamente.


## 5. Conexión a la Base de Datos

Nos conectaremos a la base de datos que creamos en la actividad anterior.

In [5]:
DB_NAME = '../dragonball_characters.db' # Ajusta la ruta si es necesario

try:
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    print(f"Conexión a la base de datos '{DB_NAME}' establecida con éxito.")
except sqlite3.Error as e:
    print(f"Error al conectar a la base de datos: {e}")

Conexión a la base de datos '../dragonball_characters.db' establecida con éxito.


## 6. Añadir Nueva Columna a la Tabla

Añadiremos una nueva columna `ai_description` a nuestra tabla `characters` para almacenar el texto generado por la IA.

In [6]:
try:
    cursor.execute("ALTER TABLE characters ADD COLUMN ai_description TEXT")
    conn.commit()
    print("Columna 'ai_description' añadida a la tabla 'characters'.")
except sqlite3.OperationalError as e:
    if "duplicate column name" in str(e):
        print("La columna 'ai_description' ya existe.")
    else:
        print(f"Error: {e}")

Columna 'ai_description' añadida a la tabla 'characters'.


## 7. Obtención de Personajes de la Base de Datos

Obtendremos los nombres y razas de los personajes para crear los prompts.

In [7]:
cursor.execute("SELECT id, name, race FROM characters")
characters_to_enrich = cursor.fetchall()

In [10]:
import google.generativeai as genai

for model in genai.list_models():
    if "generateContent" in model.supported_generation_methods:
        print(model.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models

## 8. Interactuando con la API de Gemini

Ahora, para cada personaje, crearemos un prompt, llamaremos a la API de Gemini y actualizaremos la base de datos con la historia generada.

In [11]:
# model = genai.GenerativeModel('gemini-pro')
# model = genai.GenerativeModel("gemini-1.5-flash")
model = genai.GenerativeModel("gemini-2.5-flash")

for char_id, name, race in characters_to_enrich:
    print(f"Generando historia para {name}...")
    prompt = f"Escribe una breve historia creativa y emocionante para el personaje de Dragon Ball llamado {name}, que es de la raza {race}."
    try:
        response = model.generate_content(prompt)
        ai_description = response.text
        cursor.execute("UPDATE characters SET ai_description = ? WHERE id = ?", (ai_description, char_id))
        conn.commit()
        print(f"  -> Historia generada y guardada para {name}.")
    except Exception as e:
        print(f"  -> Error al generar historia para {name}: {e}")

print("\nProceso de enriquecimiento completado.")

Generando historia para Zarbon...
  -> Historia generada y guardada para Zarbon.
Generando historia para Dodoria...
  -> Historia generada y guardada para Dodoria.
Generando historia para Ginyu...
  -> Historia generada y guardada para Ginyu.
Generando historia para Celula...
  -> Historia generada y guardada para Celula.
Generando historia para Gohan...
  -> Historia generada y guardada para Gohan.

Proceso de enriquecimiento completado.


## 9. Visualización de los Datos Enriquecidos

Finalmente, consultemos la base de datos para ver los resultados.

In [12]:
print("--- Personajes con Descripciones de IA ---")
cursor.execute("SELECT name, ai_description FROM characters")
enriched_characters = cursor.fetchall()

for name, ai_desc in enriched_characters:
    print(f"\n**{name}**")
    print(ai_desc)

--- Personajes con Descripciones de IA ---

**Zarbon**
¡Absolutamente! Aquí tienes una historia emocionante y un poco introspectiva para Zarbon, imaginándolo como parte de la fría y calculadora raza de Frieza:

---

**El Sacrificio de la Elegancia**

La nave de combate se deslizó silenciosamente a través del velo púrpura de la nebulosa de Cygnus X-1, su destino: el planeta Xylos-7, una joya verde esmeralda y azul que se negaba a inclinarse ante el Imperio de Frieza. Dentro de la cámara de mando, la figura elegante y esbelta de Zarbon, con su piel verde esmeralda inmaculada y sus largos cabellos aquamarina, observaba la proyección holográfica del mundo rebelde. Sus ojos ámbar, de una belleza inusual, no mostraban más que fría determinación.

Como miembro de la raza de Frieza, Zarbon no solo poseía una fuerza devastadora, sino también una apreciación innata por la perfección, la estrategia y, sobre todo, la estética. Para él, la brutalidad sin gracia era una afrenta; la victoria, un arte

## 10. Conclusión y Próximos Pasos

¡Felicidades! Has aprendido a:
- Conectar con una API de IA generativa (Google Gemini).
- Crear prompts para generar texto creativo.
- Enriquecer una base de datos existente con datos generados por IA.

**Ideas para Próximos Pasos:**
- **Generación de Imágenes:** Usar un modelo de texto a imagen para crear retratos de los personajes.
- **Análisis de Sentimientos:** Analizar las descripciones originales para determinar el sentimiento (positivo, negativo, neutral).
- **Chatbot de Personajes:** Crear un chatbot que responda como si fuera un personaje de Dragon Ball.

In [13]:
if conn:
    conn.close()
    print("\nConexión a la base de datos cerrada.")


Conexión a la base de datos cerrada.


Si quieres hacerlo dinámico y que use siempre el modelo más reciente compatible, puedes listar los modelos y filtrar automáticamente uno válido:

In [ ]:
available_models = [m.name for m in genai.list_models() if "generateContent" in m.supported_generation_methods]
model_name = next((m for m in available_models if "gemini-2.5-flash" in m), "gemini-2.5-pro")
model = genai.GenerativeModel(model_name)
print(f"✅ Usando modelo: {model_name}")